So I've not made the best use of the `itertools` library in the last few challenges, but this looks like a good opportunity to use `cycle`.

I'm also going to use a pandas `Series` to give me the `argmax` method.

In [1]:
import pandas as pd

import itertools as it

Represent the input as a list of integers:

In [2]:
testCase_sr=pd.Series([0, 2, 7, 0])
testCase_sr

0    0
1    2
2    7
3    0
dtype: int64

Much as I hate OO programming, I hate writing things which look like functions that alter lists even more, so I'll subclass `pd.Series` to make a `Debugger` class:

In [3]:
class Debugger(pd.Series):
    def __init__(self, listIn):
        super().__init__(listIn)
        
    def reallocate(self):
        '''Reallocate values from the block with the largest value'''
        # Find the index of the largest value:
        maxVal_ix=self.argmax()
        
        # and the value itself
        maxVal_i=self.max()
        
        # Set up an iterator on the index of the Series, 
        # starting at the item after maxVal_ix
        idxCycler=it.dropwhile(lambda x: x!=maxVal_ix,
                               it.cycle(self.index))
        # Also, remove the value that's actually equal 
        next(idxCycler)

        # Now set the maximum value to zero:
        self[maxVal_ix]=0
        
        # And redistribute the value to the next maxVal_i blocks:
        for i in range(maxVal_i):
            self[next(idxCycler)]+=1
        
        return True
        
        

u=Debugger([0,2,7,0])
print((u.argmax(), u.max()))
u.reallocate()
u

(2, 7)


0    2
1    4
2    1
3    2
dtype: int64

In [4]:
u.reallocate()
u

0    3
1    1
2    2
3    3
dtype: int64

In [5]:
u.reallocate()
u

0    0
1    2
2    3
3    4
dtype: int64

Seems to be working OK. So let's define a function to find the number of cycles before a repetition. For each visited state, I'll store it in a set as a tuple, `Debugger` instances being unhashable.

In [6]:
def steps_until_repetition(listIn_ls):
    '''Return the number of iterations before we see a repeated state'''
    myCase_debug=Debugger(listIn_ls)
    visitedStates_set=set()
    
    iterations_i=0
    while tuple(myCase_debug) not in visitedStates_set:
        visitedStates_set.add(tuple(myCase_debug))
        myCase_debug.reallocate()
        iterations_i += 1
        
    return iterations_i
        

In [7]:
assert steps_until_repetition([0, 2, 7, 0])==5

Now try with the test case:

In [8]:
with open('data/day6.txt') as fIn:
    test_ls=[int(x) for x in fIn.read().split()]
test_ls

[0, 5, 10, 0, 11, 14, 13, 4, 11, 8, 8, 7, 1, 4, 12, 11]

In [9]:
steps_until_repetition(test_ls)

7864